# Test FRED data for macros

In [1]:
%pip install fredapi

Note: you may need to restart the kernel to use updated packages.


In [2]:
from fredapi import Fred
import pandas as pd
import datetime
import numpy as np
import pickle
fred = Fred(api_key='667bd18161dd7c15d25260319fe6b425')

In [3]:
date_start = '2000-01-01'
date_end = '2018-03-01'
ts = [
    'CSUSHPINSA',
    'A191RL1Q225SBEA',
    'A191RP1Q027SBEA',
    'A067RL1Q156SBEA',
    'A067RP1Q027SBEA',
    'UNRATE',
    'CPALTT01USQ659N',
    'TB3MS',
    'GS5',
    'GS10',
    'BAMLC0A4CBBBEY',
    'MORTGAGE30US',
    'MPRIME',
    'SP500',
    'CSUSHPINSA',
    'COMREPUSQ159N',
    'VIXCLS',
]
df = { n : fred.get_series(n, observation_start=date_start, observation_end=date_end) for n in ts }

In [4]:
dr = pd.date_range(date_start, date_end, freq='MS')
fred_ds = pd.DataFrame(df, dr).fillna(method='ffill').fillna(method='bfill')
fred_ds = fred_ds.drop(columns=['SP500', 'COMREPUSQ159N']) # not full series at FRED

In [5]:
fred_ds_norm = ((fred_ds-fred_ds.mean())/fred_ds.std())

In [6]:
def yyyymm2pstr(yyyymm):
    yyyymm_str = str(yyyymm)
    return yyyymm_str[:4] + '-' + yyyymm_str[4:]

yyyymm_start = 200002
yyyymm_end = 200101
fred_ds_norm[yyyymm2pstr(yyyymm_start):yyyymm2pstr(yyyymm_end)]

,CSUSHPINSA,A191RL1Q225SBEA,A191RP1Q027SBEA,A067RL1Q156SBEA,A067RP1Q027SBEA,UNRATE,CPALTT01USQ659N,TB3MS,GS5,GS10,BAMLC0A4CBBBEY,MORTGAGE30US,MPRIME,VIXCLS
2000-02-01,-2.141428,-0.215852,0.067360,1.335810,1.640903,-1.126563,0.850189,2.139450,2.549218,2.401848,1.726593,1.927219,1.993994,0.343515
2000-03-01,-2.105518,-0.215852,0.067360,1.335810,1.640903,-1.183292,0.850189,2.215057,2.430188,2.193564,1.732727,1.927219,2.045070,0.154019
2000-04-01,-2.062435,2.305300,2.220918,0.480202,0.447257,-1.296752,0.921520,2.198856,2.271481,1.977269,1.732727,1.927219,2.131900,0.154019
2000-05-01,-2.015865,2.305300,2.220918,0.480202,0.447257,-1.183292,0.921520,2.269062,2.555831,2.337761,1.971921,1.927219,2.254484,0.597921
2000-06-01,-1.969616,2.305300,2.220918,0.480202,0.447257,-1.183292,0.921520,2.215057,2.297932,2.065390,2.033253,1.927219,2.387283,0.229399
2000-07-01,-1.934908,-0.636044,-0.435137,0.530532,0.657900,-1.183292,1.064552,2.360870,2.218578,2.025335,2.033253,1.927219,2.387283,0.229399
2000-08-01,-1.902887,-0.636044,-0.435137,0.530532,0.657900,-1.126563,1.064552,2.431076,2.139225,1.849095,1.775659,1.927219,2.387283,0.039903
2000-09-01,-1.878279,-0.636044,-0.435137,0.530532,0.657900,-1.240022,1.064552,2.382472,2.053258,1.825062,1.671395,1.927219,2.387283,-0.276272
2000-10-01,-1.854513,0.204340,0.246823,-0.299911,-0.184674,-1.240022,0.999760,2.441877,1.954066,1.776997,1.671395,1.927219,2.387283,-0.276272
2000-11-01,-1.831989,0.204340,0.246823,-0.299911,-0.184674,-1.240022,0.999760,2.474280,1.901164,1.760975,1.849257,1.927219,2.387283,0.430411


In [7]:
fred_ds_norm_np = fred_ds_norm.to_numpy(dtype=np.float32)
fred_ds_norm_np.shape

(219, 14)

In [8]:
fred_ds_norm['yyyymm']=fred_ds_norm.index.strftime('%Y%m').astype(int)

In [9]:
ym2idx = {ym:i for i, ym in enumerate(fred_ds_norm.yyyymm)}

In [10]:
ys , ye = 200002, 200003

In [11]:
fred_ds_norm_np[ym2idx[ys]:(ym2idx[ye]+1)]

array([[-2.1414278 , -0.21585202,  0.06736016,  1.3358104 ,  1.6409035 ,
        -1.1265627 ,  0.8501892 ,  2.1394503 ,  2.5492182 ,  2.4018478 ,
         1.7265935 ,  1.9272194 ,  1.9939935 ,  0.34351483],
       [-2.1055183 , -0.21585202,  0.06736016,  1.3358104 ,  1.6409035 ,
        -1.1832924 ,  0.8501892 ,  2.215057  ,  2.4301877 ,  2.1935642 ,
         1.7327267 ,  1.9272194 ,  2.0450702 ,  0.15401924]],
      dtype=float32)

In [12]:
## Save FRED and YYYYMM2IDX

In [13]:
np.save('/home/user/notebooks/data/train/fred_norm.npy', fred_ds_norm_np)
np.save('/home/user/notebooks/data/valid/fred_norm.npy', fred_ds_norm_np)
np.save('/home/user/notebooks/data/test/fred_norm.npy', fred_ds_norm_np)

In [14]:
with open('/home/user/notebooks/data/train/ym2idx.pickle', 'wb') as f:
    pickle.dump(ym2idx, f, pickle.HIGHEST_PROTOCOL)

with open('/home/user/notebooks/data/valid/ym2idx.pickle', 'wb') as f:
    pickle.dump(ym2idx, f, pickle.HIGHEST_PROTOCOL)

with open('/home/user/notebooks/data/test/ym2idx.pickle', 'wb') as f:
    pickle.dump(ym2idx, f, pickle.HIGHEST_PROTOCOL)